In [2]:
import pandas as pd
import numpy as np

In [5]:
biib = pd.read_csv('./Data/BIIB.OQ.csv')
bio = pd.read_csv('./Data/BIO.N.csv')
lkq = pd.read_csv('./Data/LKQ.OQ.csv')
mktx = pd.read_csv('./Data/MKTX.OQ.csv')
txt = pd.read_csv('./Data/TXT.N.csv')
vrsk = pd.read_csv('./Data/VRSK.OQ.csv')

In [9]:
vrsk.isna().sum()

Unnamed: 0               0
Instrument               0
Date                    69
1 Month Total Return    69
Company Market Cap      69
Price Close             69
dtype: int64

Estimate historical mean and variance covariance matrix of the stocks in your dataset
with the methods described below. Estimate the variance covariance matrix with a
Shrinkage toward Constant Correlation Approach (consider k=0.3). Estimate the vector
of averages with an exponential average (lambda=0.01). Comment on the correlations
among the stocks observed in your dataset.

Plot the efficient portfolio frontier (in the standard deviation-mean space) with a risk-
free asset (with annual return Rf=0.02). Comment on the obtained tangent portfolio (or
risky asset portfolio).

Plot the efficient portfolio frontier (in the standard deviation-mean space) with the
same risky and risk-free assets but imposing two different constraints:

    - the sum of portfolios weights of the first and second stock in your dataset is
    equal to 50%.
    - the portfolio weights are above 10% for each stocks.

Comment on the differences on the frontier portfolio with 0.5% monthly expected
return with and without the two constraints (you should consider three different
frontiers: no constraint, constraint 1 and constraint 2).

Estimate CAPM alpha and beta for your stocks, use the returns of the S&P 500 (you
should obtain returns from S&P 500 prices) as proxy for the market returns. Comment
on the results, which of the stocks has generated alpha in the previous years? Assume
Rf=0.02.

Compute market implicit return π in a Black Litterman framework and compute the market portfolio (assuming that the market comprises only 6 stocks)

Formulate at least two Black Litterman views for the returns of the stocks in your
portfolio for January 2024 (for formulating the views you can use historical averages,
news and or public balance sheet data). Comment on the Black Litterman portfolio
obtained when considering your views.